### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [129]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1
    FEATURE_THRESHOLD = 1e-7
    
    def __init__(
        self,
        min_samples_split=2,
        max_depth=None,
        sufficient_share=1.0,
        criterion='gini',
        max_features=None
    ):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return ((1 - np.sum(
            (np.bincount(l_c) / l_c.shape[0]) ** 2
        )) * l_c.shape[0] + (1 - np.sum(
            (np.bincount(r_c) / r_c.shape[0]) ** 2)
                            ) * r_c.shape[0]) / (l_c.shape[0] + r_c.shape[0])

    def __entropy(self, l_c, l_s, r_c, r_s):
        return  # Ваш код в 1 строчку

    def __misclass(self, l_c, l_s, r_c, r_s):
        return  # Ваш код в 1 строчку

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[: int(np.sqrt(feature_ids.shape[0]))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[: int(np.log(feature_ids.shape[0]) / np.log(2))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        feature_ids = self.get_feature_ids(x.shape[1])
        min_gain = 2
        for idx in feature_ids:
            col = x[:, idx]
            col, y_sort = self.__sort_samples(col, y)
            for ind in np.where(y_sort[:-1] != y_sort[1:])[0] + 1:
                l_s, r_s = col[:ind], col[ind:]
                l_c, r_c = y_sort[:ind], y_sort[ind:]
                gain = self.G_function(l_c, l_s, r_c, r_s)
                if gain == 0:
                    return idx, col[ind - 1], ind
                if gain < min_gain:
                    res_idx = idx
                    res_thr = col[ind - 1]
                    res_ind = ind
                    min_gain = gain
        return res_idx, res_thr, res_ind

    def __fit_node(self, x, y, node_id, depth):
        if depth == self.max_depth or self.min_samples_split >= x.shape[0] or len(np.unique(y)) == 1:
            self.tree[node_id] = [self.__class__.LEAF_TYPE,
                                  np.argmax(np.bincount(y))]
            return
        feature_id, threshold, split_ind = self.__find_threshold(x, y)
        self.tree[node_id] = [self.__class__.NON_LEAF_TYPE,
                              feature_id, threshold]
        x_tmp = x[x[:, feature_id].argsort()]
        y_tmp = y[x[:, feature_id].argsort()]
        X_left, X_right, y_left, y_right = x_tmp[split_ind:], x_tmp[:split_ind], y_tmp[split_ind:], y_tmp[:split_ind]
#        print(y_left, y_right, threshold, x[:, feature_id])
        self.__fit_node(X_left, y_left, 2 * node_id + 1, depth + 1)
        self.__fit_node(X_right, y_right, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [78]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_features='sqrt')
clf = DecisionTreeClassifier(min_samples_split=2, max_features='sqrt')

In [72]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [103]:
%time clf.fit(X_train, y_train)

CPU times: user 3.86 ms, sys: 38 µs, total: 3.9 ms
Wall time: 4.43 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [104]:
%time my_clf.fit(X_train, y_train)

CPU times: user 52.8 ms, sys: 8.02 ms, total: 60.8 ms
Wall time: 58.1 ms


## Проверка качества работы на wine

In [89]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.890993265993266

In [90]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.94747474747474747

## Подготовка данных Speed Dating Data 

In [105]:
from sklearn import preprocessing

In [106]:
Data = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', sep=',', encoding="ISO-8859-1")

In [107]:
Data_processed = Data.copy()

In [108]:
for feat in Data.columns: 
    if Data[feat].dtype == 'object':
        Data_processed[feat] = Data[[feat]].fillna('0').apply(preprocessing.LabelEncoder().fit_transform)[feat]

In [109]:
Data_processed.fillna(0, inplace=True)

In [110]:
target = 'match'

In [133]:
train, test = train_test_split(Data_processed)

In [120]:
clf = DecisionTreeClassifier()

In [121]:
%time clf.fit(train.drop(target, axis=1), train[target])

CPU times: user 53.2 ms, sys: 16 ms, total: 69.3 ms
Wall time: 68.2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [122]:
f1_score(clf.predict(np.array(test.drop(target, axis=1))), np.array(test[target]))

1.0

In [181]:
my_clf = MyDecisionTreeClassifier()

In [182]:
%time my_clf.fit(np.array(train.drop(target, axis=1)), np.array(train[target]))

0
cycle: 0
cycle: 1
cycle: 2
cycle: 3
cycle: 4
cycle: 5
cycle: 6
cycle: 7
cycle: 8
cycle: 9
cycle: 10
cycle: 11
cycle: 12
cycle: 13
cycle: 14
cycle: 15
cycle: 16
cycle: 17
cycle: 18
cycle: 19
cycle: 20
cycle: 21
cycle: 22
cycle: 23
cycle: 24
cycle: 25
cycle: 26
cycle: 27
cycle: 28
cycle: 29
cycle: 30
cycle: 31
cycle: 32
cycle: 33
cycle: 34
cycle: 35
cycle: 36
cycle: 37
cycle: 38
cycle: 39
cycle: 40
cycle: 41
cycle: 42
cycle: 43
cycle: 44
cycle: 45
cycle: 46
cycle: 47
cycle: 48
cycle: 49
cycle: 50
cycle: 51
cycle: 52
cycle: 53
cycle: 54
cycle: 55
cycle: 56
cycle: 57
cycle: 58
cycle: 59
cycle: 60
cycle: 61
cycle: 62
cycle: 63
cycle: 64
cycle: 65
cycle: 66
cycle: 67
cycle: 68
cycle: 69
cycle: 70
cycle: 71
cycle: 72
cycle: 73
cycle: 74
cycle: 75
cycle: 76
cycle: 77
cycle: 78
cycle: 79
cycle: 80
cycle: 81
cycle: 82
cycle: 83
cycle: 84
cycle: 85
cycle: 86
cycle: 87
cycle: 88
cycle: 89
cycle: 90
cycle: 91
cycle: 92
cycle: 93
cycle: 94
cycle: 95
cycle: 96
cycle: 97
cycle: 98
cycle: 99
cycle: 1

In [177]:
f1_score(my_clf.predict(np.array(test.drop(target, axis=1))), np.array(test[target]))

1.0

In [179]:
my_clf.tree

{0: [0, 22, 0.0], 1: [0, 96, 0.0], 2: [1, 0], 3: [1, 1], 4: [1, 0]}

In [140]:
nd_id = 4

In [169]:
np.isin(np.array([1, 2, 3]), np.array([1]))

array([ True, False, False], dtype=bool)

In [187]:
np.ones(6).reshape(2, -1)[:2]

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.]])

In [180]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1
    FEATURE_THRESHOLD = 1e-7
    
    def __init__(
        self,
        min_samples_split=2,
        max_depth=None,
        sufficient_share=1.0,
        criterion='gini',
        max_features=None
    ):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return ((1 - np.sum(
            (np.bincount(l_c) / l_c.shape[0]) ** 2
        )) * l_c.shape[0] + (1 - np.sum(
            (np.bincount(r_c) / r_c.shape[0]) ** 2)
                            ) * r_c.shape[0]) / (l_c.shape[0] + r_c.shape[0])

    def __entropy(self, l_c, l_s, r_c, r_s):
        return  # Ваш код в 1 строчку

    def __misclass(self, l_c, l_s, r_c, r_s):
        return  # Ваш код в 1 строчку

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[: int(np.sqrt(feature_ids.shape[0]))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[: int(np.log(feature_ids.shape[0]) / np.log(2))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y, node_id):
        print(node_id)
        feats = self._feats[max(0, (node_id - 1) // 2)]
        np.random.shuffle(feats)
        #feature_ids = self.get_feature_ids()
        constf = - np.ones(x.shape[1])
        ctr = 0
        min_gain = 2
        cycle = 0
        for idx in feats:
            print(f"cycle: {cycle}")
            cycle += 1
            col = x[:, idx]
            col, y_sort = self.__sort_samples(col, y)
            if col[-1] <= col[0] + self.__class__.FEATURE_THRESHOLD:
                constf[ctr] = idx
                ctr += 1
                continue
            for ind in np.where(y_sort[:-1] != y_sort[1:])[0] + 1:
                l_s, r_s = col[:ind], col[ind:]
                l_c, r_c = y_sort[:ind], y_sort[ind:]
                gain = self.G_function(l_c, l_s, r_c, r_s)
                if gain == 0:
                    self._feats[node_id] = feats[~np.isin(feats, constf)]
                    return idx, col[ind - 1], ind
                if gain < min_gain:
                    res_idx = idx
                    res_thr = col[ind - 1]
                    res_ind = ind
                    min_gain = gain
        self._feats[node_id] = feats[~np.isin(feats, constf)]
        return res_idx, res_thr, res_ind

    def __fit_node(self, x, y, node_id, depth):
        if depth == self.max_depth or self.min_samples_split >= x.shape[0] or len(np.unique(y)) == 1:
            self.tree[node_id] = [self.__class__.LEAF_TYPE,
                                  np.argmax(np.bincount(y))]
            return
        feature_id, threshold, split_ind = self.__find_threshold(x, y, node_id)
        self.tree[node_id] = [self.__class__.NON_LEAF_TYPE,
                              feature_id, threshold]
        x_tmp = x[x[:, feature_id].argsort()]
        y_tmp = y[x[:, feature_id].argsort()]
        X_left, X_right, y_left, y_right = x_tmp[split_ind:], x_tmp[:split_ind], y_tmp[split_ind:], y_tmp[:split_ind]
#        print(y_left, y_right, threshold, x[:, feature_id])
        self.__fit_node(X_left, y_left, 2 * node_id + 1, depth + 1)
        self.__fit_node(X_right, y_right, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self._feats = {0: np.arange(x.shape[1])}
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


## Проверка скорости работы на Speed Dating Data 

In [ ]:
# тут должен быть код типа %time clf.fit(X_train, y_train)

In [ ]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)

## Проверка качества работы на Speed Dating Data

In [ ]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

In [ ]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

## Задание 3

## Задание 4

## Задание 5